In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm
from os.path import join, isfile
from os import listdir
from functools import partial

import glam
import parameter_recovery as pr

%load_ext autoreload
%autoreload 2

/Users/felixmolter/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Systematic parameter recovery – Hierarchical model, with Metropolis

In [2]:
output_folder = join('results', 'parameter-recovery_hierarchical', 'metropolis')
label = 'hierarchical_metropolis'

## Define parameter ranges from 2011 estimates

In [3]:
bounds = dict(v=[1e-10, 0.0002],
              gamma=[-10, 1],
              s=[1e-10, 0.05],
              tau=[0., 5])

In [4]:
means = {'v': np.array([3.30e-05, 8.50e-05, 1.37e-04]),
         'gamma': np.array([-0.7,  0,  0.7]),
         's': np.array([0.0067, 0.0095, 0.0123]),
         'tau': np.array([0.215, 0.675, 1.135])}
means

{'v': array([3.30e-05, 8.50e-05, 1.37e-04]),
 'gamma': array([-0.7,  0. ,  0.7]),
 's': array([0.0067, 0.0095, 0.0123]),
 'tau': array([0.215, 0.675, 1.135])}

In [5]:
sds = dict(v=0.00001,
           s=0.001,
           gamma=0.2,
           tau=0.2)

In [6]:
parameter_info = {parameter: dict(mu=dict(low=means[parameter][0],
                                          medium=means[parameter][1],
                                          high=means[parameter][2]),
                                  sd=sds[parameter],
                                  bounds=tuple(bounds[parameter]))
                  for parameter in ['v', 'gamma', 's', 'tau']}

## Run the recovery in parallel

In [7]:
partial_output_folder = join(output_folder, 'partial')
traceplot_folder = join(partial_output_folder, 'traceplots')

simulate_group_args = dict(kind='hierarchical',
                           n_individuals=20,
                           n_trials=50, 
                           n_items=3,
                           error_weight=0,
                           value_range=[1, 10],
                           seed=None)

make_model_args = dict(kind='hierarchical',
                       t0_val=0,
                       error_weight=0)

fit_args = dict(method='MCMC',
                step=pm.Metropolis,
                draws=20000,
                tune=180000,
                cores=1,
                chains=2,
                progressbar=False)

wrapper = partial(pr.recover_and_save_hierarchical,
                  output_folder=partial_output_folder,
                  label=label,
                  simulate_group_args=simulate_group_args,
                  make_model_args=make_model_args,
                  fit_args=fit_args,
                  save_trace_to=None,
                  save_traceplot_to=traceplot_folder)

In [8]:
from multiprocessing import Pool

n_cores = 22
p = Pool(n_cores)

output = p.map(wrapper, pr.generate_hierarchical_parameter_sets(parameter_info))

Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchi

INFO (theano.gof.compilelock): Waiting for existing lock by process '4590' (I am process '4585')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '4590' (I am process '4586')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '4590' (I am process '4587')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '4590' (I am process '4601')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i3

Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


INFO (theano.gof.compilelock): Waiting for existing lock by process '4585' (I am process '4586')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '4585' (I am process '4587')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '4585' (I am process '4601')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '4585' (I am process '4604')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i3

Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


...

/!\ Automatically setting parameter precision...


The estimated number of effective samples is smaller than 200 for some parameters.


/!\ Automatically setting parameter precision...


The estimated number of effective samples is smaller than 200 for some parameters.


/!\ Automatically setting parameter precision...


## Combine results

In [9]:
filenames = [file for file in listdir(partial_output_folder)
             if file.endswith('.csv')]

partial_recoveries = []

for file in filenames:
    partial = pd.read_csv(join(partial_output_folder, file))
    partial_recoveries.append(partial)

parameter_recovery = pd.concat(partial_recoveries).sort_values('index').reset_index(drop=True)
del partial_recoveries
parameter_recovery.to_csv(join(output_folder, 'parameter-recovery_{}.csv'.format(label)))
parameter_recovery.head()

,index,v_level,gamma_level,s_level,tau_level,v_gen,v_gen_actual,gamma_gen,gamma_gen_actual,s_gen,...,gamma_rec,gamma_rec_hpd_lower,gamma_rec_hpd_upper,s_rec,s_rec_hpd_lower,s_rec_hpd_upper,tau_rec,tau_rec_hpd_lower,tau_rec_hpd_upper,converged
0,0,low,low,low,low,0.000033,0.000039,-0.7,-0.779017,0.0067,...,-0.766530,-0.969478,-0.573529,0.006564,0.005744,0.007390,0.118701,0.011519,0.210938,True
1,1,low,low,low,medium,0.000033,0.000032,-0.7,-0.659172,0.0067,...,-0.587455,-0.650783,-0.514818,0.006696,0.006254,0.007152,0.765726,0.627700,0.916429,True
2,2,low,low,low,high,0.000033,0.000031,-0.7,-0.685173,0.0067,...,-0.582108,-0.637719,-0.526333,0.006384,0.005944,0.006866,1.218035,0.948798,1.495412,True
3,3,low,low,medium,low,0.000033,0.000032,-0.7,-0.766911,0.0095,...,-0.684036,-0.889326,-0.489667,0.008897,0.008242,0.009547,0.310423,0.221002,0.396320,True
4,4,low,low,medium,medium,0.000033,0.000033,-0.7,-0.714497,0.0095,...,-0.633952,-0.727031,-0.548031,0.009490,0.008759,0.010277,0.888521,0.617116,1.191125,True
